In [122]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from math import sqrt 

raw_train  = pd.read_csv("../Assignment1 Dataset/Datasets/q3/train.csv")
raw_test = pd.read_csv("../Assignment1 Dataset/Datasets/q3/test.csv")
raw_test_labels = pd.read_csv("../Assignment1 Dataset/Datasets/q3/test_labels.csv", header= None)


In [107]:
raw_test_labels.head()

,0,1
0,1001,128000
1,1002,124900
2,1003,191000
3,1004,144000
4,1005,231500


In [12]:
# Feature Engineering

def get_categorical_columns(dataframe): 
    cat_columns = list(set(dataframe.columns)  - set(dataframe._get_numeric_data().columns)) 
    return dataframe[cat_columns ]

def feature_processing(raw_train):
    
    # drop id column 
    raw_train.drop(['Id'] , axis =1  , inplace= True )
    
    # filter out columns with missing values more than 80%
    columns_subset = raw_train.columns[raw_train.isnull().mean()<0.8]
    raw_train = raw_train[columns_subset]
    
    # process cateogorical data 
    raw_train['FireplaceQu'] = raw_train['FireplaceQu'].fillna('missing')
    cols = ['GarageQual'   ,
            'GarageFinish' ,
            'GarageType'   ,
            'BsmtFinType2' ,
            'BsmtExposure' ,
            'BsmtFinType1' ,
            'BsmtCond'     ,
            'BsmtQual'     ,
            'MasVnrType'   ,
            'Electrical'   , 
            'GarageCond'   ]

    raw_train[cols]=raw_train[cols].fillna(raw_train.mode().iloc[0])
    # process numeric data 
    numeric_train = raw_train._get_numeric_data().fillna(raw_train._get_numeric_data().mean())
    cat_train  = get_categorical_columns(raw_train)
    # combine th data 
    dataset = pd.concat([ numeric_train , cat_train.reindex(numeric_train.index)] , axis = 1)
    print(dataset.isnull().shape)
    return dataset

    
# feature engineering 

def implement_label_encoding(train_dataset):
    
    #implement label encoding 
    cols_to_labelencode  = [
    'KitchenAbvGr'  ,   
    'BsmtHalfBath'  ,    
    'HalfBath'      ,  
    'BsmtFullBath'  , 
    'FullBath'      ,
    'Fireplaces'    ,
    'GarageCars'    ,
    'BedroomAbvGr'   ]

    for column in cols_to_labelencode:
        train_dataset[column] = train_dataset[column].astype('category').cat.codes
 
    
    return train_dataset

train_dataset  = feature_processing(raw_train)
train_cleaned_dataset = implement_label_encoding(train_dataset)

C:\Users\sudha\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sudha\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


(1000, 76)


In [17]:
train_encoded = pd.get_dummies(train_cleaned_dataset)
train_encoded.shape

(1000, 273)

# Building a baseline model - mean 

In [134]:
# Building a baseline model - mean 
h = np.ones(1000)*y.mean()

MSE = np.square(np.subtract(y_baseline,h)).mean() 
print("MSE " , MSE)
print("MAE " , metrics.mean_absolute_error(y_baseline , h))
print('R square ',metrics.r2_score(y_baseline, h))

MSE  6765036278.128341
MAE  59406.06616
R square  0.0


# Building a baseline model sklearn

In [21]:
# building a baseline model 
def rmse(h,y):
    m = len(h)
    mse = np.sum(list(((h-y)**2)))*(1/m)
    return sqrt(mse)

X_baseline , y_baseline  = train_encoded.loc[:, train_encoded.columns != 'SalePrice'] , train_encoded['SalePrice']
from sklearn.tree import DecisionTreeClassifier
RANDOM_SEED = 42 
reg = DecisionTreeClassifier(min_samples_leaf= 5  , random_state = RANDOM_SEED )
reg.fit(X_baseline , y_baseline )

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [22]:
# measuring baseline model performance
h = reg.predict(X_baseline)

print(rmse(h,y_baseline))
print("built in method : " , sqrt(mean_squared_error(h ,y_baseline)))
print("coded method ", rmse(h,y_baseline))
# print R square for model 
print('R square ',metrics.r2_score(y_baseline, h))

58710.04050263464
built in method :  58710.04050263464
coded method  58710.04050263464
R square  0.49048774993789046


In [65]:
#reg.feature_importances_.shape
important_features = dict(zip(X_baseline.columns, reg.feature_importances_))
{k: v for k, v in sorted(important_features.items(), key=lambda item: item[1] , reverse  = True)}

{'LotArea': 0.06384825251008615,
 'MoSold': 0.06238616796697659,
 'GarageArea': 0.05819966729825631,
 'BsmtFinSF1': 0.04767422711961682,
 'TotalBsmtSF': 0.04526114077619708,
 'BsmtUnfSF': 0.04233847993739594,
 'WoodDeckSF': 0.042183271183145174,
 'YearRemodAdd': 0.039225038106392496,
 'GrLivArea': 0.03844666347097142,
 '1stFlrSF': 0.036952113155795115,
 'LotFrontage': 0.0354463263089644,
 'OpenPorchSF': 0.03057506734682145,
 'YearBuilt': 0.027697681934636367,
 'MasVnrArea': 0.017131305639961625,
 'MSSubClass': 0.016072449046832617,
 'BsmtFinType1_BLQ': 0.015144905786318394,
 'OverallCond': 0.012838722786241145,
 'GarageYrBlt': 0.012789137496817841,
 'BedroomAbvGr': 0.012698850425043876,
 'BsmtFinSF2': 0.01268038569641291,
 'LotShape_Reg': 0.012582232819581447,
 'SaleType_COD': 0.012368465828047407,
 'EnclosedPorch': 0.011999574520581096,
 'SaleCondition_Normal': 0.011911665549734347,
 'TotRmsAbvGrd': 0.011865355191146842,
 'GarageFinish_Unf': 0.011076530326690242,
 'GarageFinish_RFn': 

Because of many features used the decision tree does many spits and is not performing well
Using feature importance to find the most important ones. 

In [82]:
# Feature Selection by feature importance criteria
features_selected = ['LotArea',
 'MoSold',
 'GarageArea',
 'BsmtFinSF1',
 'TotalBsmtSF',
 'BsmtUnfSF',
 'WoodDeckSF',
 'YearRemodAdd',
 'GrLivArea',
 '1stFlrSF',
 'LotFrontage']

In [83]:
X_baseline[features_selected].shape

# top 11 features from train data are taken to further train the model 

(1000, 11)

In [84]:
# rebuilding the baseline model 
X_baseline = X_baseline[features_selected]
reg = DecisionTreeClassifier(min_samples_leaf= 5  , random_state = RANDOM_SEED )
reg.fit(X_baseline , y_baseline )

h = reg.predict(X_baseline)

print(rmse(h,y_baseline))
print("built in method : " , sqrt(mean_squared_error(h ,y_baseline)))
print("coded method ", rmse(h,y_baseline))
# print R square for model 
print('R square ',metrics.r2_score(y_baseline, h))

62854.98126735064
built in method :  62854.98126735064
coded method  62854.98126735064
R square  0.4160048065238159


In [123]:
# baseline test performance
test_dataset  = feature_processing(raw_test)
test_dataset

C:\Users\sudha\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sudha\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


(460, 75)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageType,GarageCond,FireplaceQu,ExterQual,RoofStyle,Neighborhood,BsmtFinType1,RoofMatl,CentralAir,Condition1
0,20,80.000000,9920,5,5,1954,1954,110.0,354,290,...,Attchd,TA,missing,TA,Gable,NAmes,Rec,CompShg,Y,Norm
1,20,60.000000,7200,5,7,1972,1972,0.0,632,0,...,Detchd,TA,missing,TA,Gable,CollgCr,ALQ,CompShg,Y,Norm
2,20,93.000000,9382,7,5,1999,2000,125.0,0,0,...,Attchd,TA,missing,Gd,Gable,CollgCr,Unf,CompShg,Y,Norm
3,90,64.000000,6979,6,5,1980,1980,0.0,1056,0,...,Detchd,TA,missing,TA,Gable,OldTown,GLQ,CompShg,Y,Norm
4,60,65.000000,15426,6,5,1997,1997,0.0,821,0,...,Attchd,TA,missing,TA,Gable,CollgCr,GLQ,CompShg,Y,Norm
5,20,67.284574,8475,4,7,1956,1956,0.0,228,0,...,Detchd,TA,missing,TA,Gable,NAmes,ALQ,CompShg,Y,Norm
6,20,90.000000,7407,6,7,1957,1996,0.0,600,0,...,Attchd,TA,missing,TA,Gable,OldTown,GLQ,CompShg,Y,Artery
7,50,105.000000,8470,3,2,1915,1982,0.0,0,0,...,Attchd,TA,missing,Fa,Hip,IDOTRR,Unf,CompShg,N,Feedr
8,120,34.000000,5381,6,5,2005,2005,135.0,900,0,...,Attchd,TA,Gd,Gd,Gable,NridgHt,ALQ,CompShg,Y,Norm
9,80,80.000000,8816,6,7,1971,1971,80.0,504,0,...,Attchd,TA,missing,TA,Gable,ClearCr,GLQ,CompShg,Y,Norm


In [136]:
h = reg.predict(test_dataset[features_selected])

MSE = np.square(np.subtract(raw_test_labels[1],h)).mean() 
print("MSE " , MSE)
print("MAE " , metrics.mean_absolute_error(raw_test_labels[1] , h))

# sklearn model performance on test data 
print('R square ',metrics.r2_score(raw_test_labels[1], h))


MSE  3275694853.7369566
MAE  38946.37173913044
R square  0.38016732251854646


# Model Building from scratch 

In [97]:
# Model building code 

class Node:

    def __init__(self, x, y, idxs, min_leaf=20):
        self.x = x
        self.y = y
        self.idxs = idxs
        self.min_leaf = min_leaf
        self.rows = len(idxs)
        self.columns = x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.variable_split()

    def variable_split(self):
        for c in range(self.columns): 
            self.best_split(c)
        if self.is_leaf: return
        x = self.split_col
        left_array = np.nonzero(x <= self.node_split_val)[0]
        right_array = np.nonzero(x > self.node_split_val)[0]
        self.left_node = Node(self.x, self.y, self.idxs[left_array], self.min_leaf)
        self.right_node = Node(self.x, self.y, self.idxs[right_array], self.min_leaf)

    def best_split(self, var_idx):

        x = self.x.values[self.idxs, var_idx]
 
        # for categorical data
        if self.x.iloc[:,var_idx].dtype == 'O':
            
            # count frequency and calculate scores 
            unique_vals = list(set(x))
            for v in unique_vals : 
                # seperate the num of cat and non-cat
                in_cat = np.array([val== v for val in x])
                not_in_cat = np.array([val!=v for val in x  ])
                split_score = self.variance_score(in_cat , not_in_cat)
                self.update_scores(split_score , v , var_idx)
                
        # for numerical data
        else : 
            for r in range(self.rows):
                left_node = x <= x[r]
                right_node = x > x[r]
                if right_node.sum() < self.min_leaf or left_node.sum() < self.min_leaf: continue

                curr_score = self.variance_score(left_node, right_node)
                self.update_scores(curr_score , x[r], var_idx)
                
                    
    def update_scores(self, current_score, nodeplit_val ,var_idx):
        
        if current_score < self.score:
            self.var_idx = var_idx
            self.score = current_score
            self.node_split_val = nodeplit_val

    def variance_score(self, lhs, rhs):
        y = self.y[self.idxs]
        lhs_std = y[lhs].std()
        rhs_std = y[rhs].std()
        return lhs_std * lhs.sum() + rhs_std * rhs.sum()

    @property
    def split_col(self):
        return self.x.values[self.idxs, self.var_idx]

    @property
    def is_leaf(self):
        return self.score == float('inf')

    def predict(self, x):
        return np.array([self.predict_instance(xi) for xi in x])

    def predict_instance(self, xi):
        if self.is_leaf: return self.val
        node = self.left_node if xi[self.var_idx] <= self.node_split_val else self.right_node
        return node.predict_instance(xi)
    
class DecisionTreeRegressor:

    def build_tree(self, X, y, min_leaf=5):
        self.dtree = Node(X, y, np.array(np.arange(len(y))), min_leaf)
        return self

    def predict(self, X):
        return self.dtree.predict(X.values)

X = raw_train.loc[:, raw_train.columns != 'SalePrice']
y =raw_train['SalePrice']

regressor = DecisionTreeRegressor().build_tree(X[['LotArea' , 'MoSold' , 'GarageArea' ,'BsmtFinSF1','TotalBsmtSF']], y)
preds = regressor.predict(X[['LotArea' , 'MoSold' , 'GarageArea' ,'BsmtFinSF1','TotalBsmtSF']])
print(preds)


[344161.66666667 413383.         167786.4        147955.5
 125500.         158960.         154319.57142857 174666.66666667
 154207.14285714 116557.14285714 109771.6         64171.42857143
 184850.         151242.85714286 119361.11111111 133577.77777778
 147985.71428571 267333.33333333 107200.         276746.16666667
 204557.14285714 151557.14285714 219200.         154319.57142857
 197233.33333333 242326.4        219566.66666667 265500.
 364351.6        192088.88888889 111900.         116375.
 126067.28571429  94286.85714286 219200.         140110.
 214139.16666667 238496.         159128.57142857 487748.33333333
 101080.         139050.         302100.         133577.77777778
 170582.71428571 232280.         221500.         152171.42857143
 148928.57142857 138000.         213300.         325120.375
 182947.5        116557.14285714 116375.         306300.
 213643.33333333 147955.5         83900.         413383.
 125500.         169058.33333333  94286.85714286 222141.66666667
 147500.    

In [98]:
h =preds
print(rmse(h,y))
print("built in method : " , sqrt(mean_squared_error(h ,y)))
print("coded method ", rmse(h,y))
# print R square for model 
print('R square ',metrics.r2_score(y, h))

35462.62472680998
built in method :  35462.62472680998
coded method  35462.62472680998
R square  0.8141033246812863


In [137]:
# model performance on test data 

test_preds = regressor.predict(raw_test[['LotArea' , 'MoSold' , 'GarageArea' ,'BsmtFinSF1','TotalBsmtSF']])

MSE = np.square(np.subtract(raw_test_labels[1],test_preds)).mean() 
print("MSE " , MSE)
print("MAE " , metrics.mean_absolute_error(raw_test_labels[1] , test_preds))


print('R square ',metrics.r2_score(raw_test_labels[1], test_preds))



MSE  3276721826.5816293
MAE  39070.39312715666
R square  0.37997299693071285


# Conclusion

In [ ]:
1. The model performs well on train set with rsquare of 81% 
2. The model performance drops on test data due to overfitting
3. The model could not be trained on all the data because of max depth recursion exceptions. to solve this we can add max depth as a criteria to control 
how deep a tree grows 
4. A large number of categories in categorical data resulted in 200+ features. Because of this the tree splits take time. 
To make it work I used top most five important features. 
5. The top features were selected based on feature importance criteria extracted. It was taken from baseline model built using sklearn library.
6. Our model works better on train data as compare to baseline model. However it overfitted. Further tuning can be done. 
7. Min samples in leaf were used as 5 in baseline model and 20 for our model built from scratch. The reason being it takes time to train in model built from scratch. I increased min samples parameters for faster model training
8. Finally, the baseline model using mean as criteria performed poorly. Sklearn model and our model built from scratch have been observed gave very similar R square on test data. This 38. 
9. The MSE and MAE were similar in sklearn and model built from scratch. It was highest for dummy baseline model 